### Librerias

In [1]:
import pandas as pd
import os
import csv

### Manipulacion de archivos

#### Funcion para obtener los directorios de entrada y salida

In [2]:
def obtener_directorios():
    """Solicita los directorios de entrada y salida al usuario."""
    directorio_entrada = input("Por favor, introduce el directorio de entrada (donde están los archivos Excel): ").strip('\"')
    directorio_salida = input("Por favor, introduce el directorio de salida (donde se guardarán los archivos CSV): ").strip('\"')

    # Normalizar las rutas para evitar problemas con \ o /
    directorio_entrada = os.path.normpath(directorio_entrada)
    directorio_salida = os.path.normpath(directorio_salida)

    return directorio_entrada, directorio_salida



#### Funcion para extraer datos de los extractos en excel originales 


In [3]:
def procesar_excel(archivo_excel):
    """Procesa un archivo Excel y devuelve un DataFrame formateado."""
    # Leer el archivo Excel
    df = pd.read_excel(archivo_excel)
    df = df.drop([0, 1, 2])  # Elimina las primeras 3 filas
    
    # Usar la primera fila como nombres de las columnas
    df.columns = df.iloc[0]
    
    # Eliminar la primera fila
    df = df[1:]
    
    # Restablecer los índices
    df.reset_index(drop=True, inplace=True)
    
    # Seleccionar las columnas deseadas
    columnas_deseadas = ['F.Valor', 'Fecha', 'Concepto', 'Movimiento', 'Importe', 'Divisa', 'Disponible', 'Observaciones']
    df = df[columnas_deseadas]
    return df


#### Funcion para converitir los df resultantes a CSV

In [4]:
def guardar_como_csv(df, archivo_csv):
    """Guarda un DataFrame como un archivo CSV."""
    df.to_csv(archivo_csv, index=False)


#### Funcion para extraer los documentos de excel del directorio original para guardarlos como CSV

In [5]:
def convertir_archivos(directorio_entrada, directorio_salida):
    """Convierte los archivos Excel en CSV."""
    for archivo in os.listdir(directorio_entrada):
        if archivo.endswith('.xlsx'):
            archivo_excel = os.path.join(directorio_entrada, archivo)
            archivo_csv = os.path.join(directorio_salida, archivo.replace('.xlsx', '.csv'))
            
            # Procesar el archivo Excel
            df = procesar_excel(archivo_excel)
            
            # Guardar el DataFrame como CSV
            guardar_como_csv(df, archivo_csv)
    
    print("Archivos convertidos y guardados en el directorio de salida.")



#### Conversion de .xlsx a CSV

In [6]:
# Ejecutar funciones necesarias
entrada, salida = obtener_directorios()
convertir_archivos(entrada, salida)

Archivos convertidos y guardados en el directorio de salida.


#### Funcion para pedir el directorio donde se encuentran los csv para unirlos

In [7]:
def obtener_directorio():
    """Solicita al usaurio el directorio donde están los archivos CSV."""
    directorio = input("Por favor, introduce el directorio donde se encuentran los archivos CSV: ").strip('\"')
    return os.path.normpath(directorio)

#### Leer todos los CSV del directorio

In [8]:
def leer_archivos_csv(directorio):
    """Lee todos los archivos CSV de un directorio y los almacena en una lista de DataFrames."""
    dataframes = []
    for archivo in os.listdir(directorio):
        if archivo.endswith('.csv'):
            archivo_csv = os.path.join(directorio, archivo)
            df = pd.read_csv(archivo_csv)
            dataframes.append(df)
    return dataframes


#### Combinar CSV en un solo dataframe

In [9]:
def combinar_dataframes(dataframes):
    """Combina una lista de DataFrames en uno solo."""
    return pd.concat(dataframes, ignore_index=True)

#### Guardar en un CSV final

In [10]:
def guardar_csv(df, ruta_salida):
    """Guarda un DataFrame combinado como un archivo CSV."""
    df.to_csv(ruta_salida, index=False)
    print(f"Archivo combinado guardado en: {ruta_salida}")


#### Ejecutar funciones para convertir CSV en un CSV final para trabajr

In [11]:
def procesar_archivos():
   #Función principal para gestionar todo el fuljo del programa
    directorio = obtener_directorio()
    dataframes = leer_archivos_csv(directorio)
    if not dataframes:
        print("No se encontraron archivos CSV en el directorio especificado.")
        return
    df_combinado = combinar_dataframes(dataframes)
    ruta_salida = input("Por favor, introduce la ruta y el nombre para el archivo CSV combinado: ").strip('\"')
    ruta_salida = os.path.normpath(ruta_salida)
    guardar_csv(df_combinado, ruta_salida)


#### Ejecutar

In [13]:
procesar_archivos()

Archivo combinado guardado en: C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Entrega de Proyecto\DocumentosUnidos.csv


### Limpieza Inicial de los datos

In [14]:
df = pd.read_csv(r'C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Proyecto\Entrega\Entrega Final\DocumentosUnidos.csv')

#### Comprobacion y limpieza

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5537 entries, 0 to 5536
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   F.Valor        5537 non-null   object 
 1   Fecha          5537 non-null   object 
 2   Concepto       5537 non-null   object 
 3   Movimiento     5463 non-null   object 
 4   Importe        5537 non-null   float64
 5   Divisa         5537 non-null   object 
 6   Divisa.1       5537 non-null   object 
 7   Disponible     5537 non-null   float64
 8   Observaciones  5537 non-null   object 
dtypes: float64(2), object(7)
memory usage: 389.4+ KB


In [16]:
#Elimino la columna Divisa.1 y Fecha, que es la Fecha de cargo, no la fecha de ejecucion del movimiento 
df = df.drop(columns=['Divisa.1', 'Fecha'])

In [17]:
#Renombro la columna F.Valor a Fecha
df = df.rename(columns={'F.Valor': 'Fecha'})

In [18]:
#Convierto la columna fecha a formato fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

#### Funciones para limpiar datos especificos

In [19]:
def limpiar_datos_observaciones(df, numeros_a_eliminar):
    for numero in numeros_a_eliminar:
        df['Observaciones'] = df['Observaciones'].str.replace(numero, '', regex=False)
    return df

def limpiar_datos_movimiento(df, numeros_a_eliminar):
    for numero in numeros_a_eliminar:
        df['Movimiento'] = df['Movimiento'].str.replace(numero, '', regex=False)
    return df

def eliminar_cargos_automaticos(df, movimientos_a_eliminar, conceptos_a_eliminar):
    for movimiento in movimientos_a_eliminar:
        df = df[~df['Movimiento'].str.contains(movimiento, na=False)]
    for concepto in conceptos_a_eliminar:
        df = df[~df['Concepto'].str.contains(concepto, na=False)]
    return df




#### Datos especificos a ser limpiados

In [ ]:
numeros_observaciones = ['''Datos a eliminar''']
numeros_movimiento = ['''Datos a eliminar''']
movimientos_a_eliminar = ['Trp redondeo tarjeta']
conceptos_a_eliminar = ['mes cuentas claras', 'Comision mensual cuentas claras']

In [21]:
# Aplicae las funciones:
df = limpiar_datos_observaciones(df, numeros_observaciones)
df = limpiar_datos_movimiento(df, numeros_movimiento)
df = eliminar_cargos_automaticos(df, movimientos_a_eliminar, conceptos_a_eliminar)

In [22]:
#Guardo el DF con la primera limpieza 
df.to_csv(r'C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Entrega de Proyecto\CSV_Limpio.csv', index=False)

### Categorizacion

#### Funcion para asignar las categorias 

In [23]:
def categorizar_concepto(concepto, dic_cat):
    """
    Recorre el diccionario de categorías buscando si alguna palabra clave está presente en el concepto.
    Si se encuentra, retorna la categoría correspondiente; de lo contrario, retorna None
    """
    for categoria, palabras in dic_cat.items():
        for palabra in palabras:
            if str(palabra) in str(concepto):  # Aseguramos que concepto sea string
                return categoria
    return None


#### Funcion pra extraer el diccionario

In [24]:
def cargar_diccionario(ruta_csv):
   
    df_dic = pd.read_csv(ruta_csv, header=None, encoding='latin8')
    df_dic.rename(columns={0: 'Categoria', 1: 'Concepto'}, inplace=True)
    # Agrupa por categoría y convierte los conceptos en una lista
    diccionario = df_dic.groupby('Categoria')['Concepto'].apply(list).to_dict()
    return diccionario


#### Funcion para extraer CSV a categorizar 

In [25]:
def cargar_datos(ruta_csv):
    df = pd.read_csv(ruta_csv)
    # Aplica lower() a cada elemento de tipo string en todas las columnas
    df = df.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))
    return df


#### Funcion para aplicar las categorias

In [26]:
def aplicar_categorizacion(df, dic_cat):
   
    df['Categoria'] = df['Concepto'].apply(lambda x: categorizar_concepto(x, dic_cat))
    df['Categoria'] = df.apply(lambda row: 'ingresos' if row['Importe'] > 0 else row['Categoria'], axis=1)
    return df


#### Funcion que aplica una serie de "reglas" extra de categorizacion especificas para cada conjunto de datos

In [27]:
def aplicar_correcciones(df, reglas):
    
    for patron, nueva_categoria in reglas.items():
        df.loc[df["Observaciones"].str.contains(patron, case=False, na=False), "Categoria"] = nueva_categoria
    # Asegurarse de que las filas con importes positivos sean categorizadas como "ingresos"
    df['Categoria'] = df.apply(lambda row: 'ingresos' if row['Importe'] > 0 else row['Categoria'], axis=1)
    return df


#### Funcion para actualizar el diccionario con categorias no registradas 

In [28]:
def actualizar_diccionario(dic_cat, conceptos, category_map):
    
    for concepto in conceptos:
        print("Concepto sin categoría:", concepto)
        C = input("Ingrese el número de la categoría correspondiente: ")
        if C in category_map:
            dic_cat[category_map[C]].append(concepto)
        else:
            print("Categoría no válida. Inténtalo de nuevo.")
    return dic_cat


#### Funcion que garda el CSV final con las categorias

In [29]:
def guardar_csv(df, ruta_csv):
    
    df.to_csv(ruta_csv, index=False)


#### Ejecucion de procesod de categorizacon

In [30]:
# Definir rutas de los archivos
ruta_diccionario = r"C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Entrega de Proyecto\DicCategorias.csv"
ruta_datos = r"C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Entrega de Proyecto\CSV_Limpio.csv"
ruta_salida = r"C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Entrega de Proyecto\ParaUsar.csv"


##### Paso 1: Cargar el diccionario de categorías

In [31]:
dic_cat = cargar_diccionario(ruta_diccionario)
print("Diccionario de categorías cargado.")

Diccionario de categorías cargado.


##### Paso 2: Cargar los datos a categorizar

In [32]:
df = cargar_datos(ruta_datos)
print("Datos cargados.")

Datos cargados.


#### Paso 3: Aplicar la categorización inicial

In [33]:
df = aplicar_categorizacion(df, dic_cat)
print("Categorización inicial aplicada.")

Categorización inicial aplicada.


##### Paso 4: Definir las reglas de corrección 

In [ ]:
reglas = {
    "Observacion extra": "viajes",
    "ahorro": "ahorro",
    "diego antonio mere caravelli": "Correction"
}



##### Paso 5: Aplicar correcciones basadas en las reglas definidas

In [35]:
df = aplicar_correcciones(df, reglas)
print("Correcciones aplicadas.")

Correcciones aplicadas.


##### Paso 6: Actualizar el diccionario para conceptos no categorizados

In [142]:
conceptos_no_categorizados = set(df[df['Categoria'].isnull()]['Concepto']) #Set de conceptos sin categoria asignada
# Mapa de números a categorías para facilitar la entrada
category_map = {
    '1': 'comida_fuera',
    '2': 'suscripcion',
    '3': 'vivienda',
    '4': 'ocio',
    '5': 'transporte',
    '6': 'impuestos',
    '7': 'viajes',
    '8': 'mascota',
    '9': 'seguros',
    '10': 'deporte',
    '11': 'prestamo',
    '12': 'compras',
    '13': 'servicios',
    '14': 'salud',
    '15': 'tecnología',
    '16': 'comida',
    '17': 'bizum',
    '18': 'amazon',
    '19': 'otros'
}
if conceptos_no_categorizados:
    print("Actualizando el diccionario con conceptos no categorizados:")
    dic_cat = actualizar_diccionario(dic_cat, conceptos_no_categorizados, category_map)
    # Vuelve a aplicar la categorización con el diccionario actualizado
    df = aplicar_categorizacion(df, dic_cat)



Actualizando el diccionario con conceptos no categorizados:
Concepto sin categoría: bam bam dner
Concepto sin categoría: æam æam
Concepto sin categoría: silûs cakes give me more
Concepto sin categoría: trady´s avda marques de a
Concepto sin categoría: park gell
Concepto sin categoría: doña jacinta
Concepto sin categoría: cafe-bar briãas
Concepto sin categoría: luis vitaliano&luis españ
Concepto sin categoría: aparcamiento easo
Concepto sin categoría: operación financiada con tarjeta
Concepto sin categoría: doæa jacinta


#### Paso 7: Guardar el DataFrame final en un archivo CSV


In [36]:
df_final = df[['Fecha', 'Importe', 'Categoria']]
guardar_csv(df_final, ruta_salida)
print("Archivo CSV guardado en", ruta_salida)

Archivo CSV guardado en C:\Users\diego\Desktop\GitHub\DiegoMere-Esp\Entrega de Proyecto\ParaUsar.csv
